<a href="https://colab.research.google.com/github/Errachidiacoder/NLP/blob/main/TP1_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
#install kaggle
!pip install -q kaggle

In [8]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"hafsadaanouni","key":"80b38cd0305adb1fd35a2473f23ec4a3"}'}

In [10]:
!mkdir -p ~/.kaggle

In [11]:
!cp kaggle.json ~/.kaggle/

In [12]:
!chmod 600 ~/.kaggle/kaggle.json

In [13]:
!kaggle datasets download -d thoughtvector/customer-support-on-twitter

 97% 164M/169M [00:01<00:00, 156MB/s]
100% 169M/169M [00:01<00:00, 137MB/s]


In [14]:
!unzip /content/customer-support-on-twitter.zip

Archive:  /content/customer-support-on-twitter.zip
  inflating: sample.csv              
  inflating: twcs/twcs.csv           


In [15]:
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from collections import Counter

In [16]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [17]:
df = pd.read_csv("sample.csv")


In [18]:
df

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,119237,105834,True,Wed Oct 11 06:55:44 +0000 2017,@AppleSupport causing the reply to be disregar...,119236,NaN
1,119238,ChaseSupport,False,Wed Oct 11 13:25:49 +0000 2017,@105835 Your business means a lot to us. Pleas...,NaN,119239.0
2,119239,105835,True,Wed Oct 11 13:00:09 +0000 2017,@76328 I really hope you all change but I'm su...,119238,NaN
3,119240,VirginTrains,False,Tue Oct 10 15:16:08 +0000 2017,@105836 LiveChat is online at the moment - htt...,119241,119242.0
4,119241,105836,True,Tue Oct 10 15:17:21 +0000 2017,@VirginTrains see attached error message. I've...,119243,119240.0
...,...,...,...,...,...,...,...
88,119330,105859,True,Wed Oct 11 13:50:42 +0000 2017,@105860 I wish Amazon had an option of where I...,119329,119331.0
89,119331,105860,True,Wed Oct 11 13:47:14 +0000 2017,They reschedule my shit for tomorrow https://t...,119330,NaN
90,119332,Tesco,False,Wed Oct 11 13:34:06 +0000 2017,"@105861 Hey Sara, sorry to hear of the issues ...",119333,119334.0
91,119333,105861,True,Wed Oct 11 14:05:18 +0000 2017,@Tesco bit of both - finding the layout cumber...,"119335,119336",119332.0


In [19]:
df.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,119237,105834,True,Wed Oct 11 06:55:44 +0000 2017,@AppleSupport causing the reply to be disregar...,119236,NaN
1,119238,ChaseSupport,False,Wed Oct 11 13:25:49 +0000 2017,@105835 Your business means a lot to us. Pleas...,NaN,119239.0
2,119239,105835,True,Wed Oct 11 13:00:09 +0000 2017,@76328 I really hope you all change but I'm su...,119238,NaN
3,119240,VirginTrains,False,Tue Oct 10 15:16:08 +0000 2017,@105836 LiveChat is online at the moment - htt...,119241,119242.0
4,119241,105836,True,Tue Oct 10 15:17:21 +0000 2017,@VirginTrains see attached error message. I've...,119243,119240.0


In [20]:
df["text_lower"] = df["text"].str.lower()
df.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id,text_lower
0,119237,105834,True,Wed Oct 11 06:55:44 +0000 2017,@AppleSupport causing the reply to be disregar...,119236,NaN,@applesupport causing the reply to be disregar...
1,119238,ChaseSupport,False,Wed Oct 11 13:25:49 +0000 2017,@105835 Your business means a lot to us. Pleas...,NaN,119239.0,@105835 your business means a lot to us. pleas...
2,119239,105835,True,Wed Oct 11 13:00:09 +0000 2017,@76328 I really hope you all change but I'm su...,119238,NaN,@76328 i really hope you all change but i'm su...
3,119240,VirginTrains,False,Tue Oct 10 15:16:08 +0000 2017,@105836 LiveChat is online at the moment - htt...,119241,119242.0,@105836 livechat is online at the moment - htt...
4,119241,105836,True,Tue Oct 10 15:17:21 +0000 2017,@VirginTrains see attached error message. I've...,119243,119240.0,@virgintrains see attached error message. i've...


In [39]:
def function_clean(text,freq_words=set(), rare_words=set()):
  # Mise en minuscules
    text = text.lower()

    # Suppression des émojis
    text = re.sub(r'(:\s?\)|:-\)|;\)|:-\()\S*', '', text)

    # Suppression des émoticônes
    text = re.sub(r'<[a-zA-Z0-9\s/]+>', '', text)

    # Suppression des balises HTML
    text = re.sub(r'<.*?>', '', text)

    # Suppression des URLs
    text = re.sub(r'https?://\S+|www\.\S+', '', text)

    # Suppression des ponctuations
    text = re.sub(r'[^\w\s]', '', text)

    # Tokenization
    tokens = word_tokenize(text)

    # Suppression des mots vides
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Suppression des mots fréquents
    tokens = [word for word in tokens if word not in freq_words]

    # Suppression des mots rares
    tokens = [word for word in tokens if word not in rare_words]

    # Stemming
    ps = PorterStemmer()
    tokens = [ps.stem(word) for word in tokens]

    # Lemmatisation
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # Reconstitution du texte
    new_text = ' '.join(tokens)

    return new_text

In [40]:
df['text_cleaned'] = df['text'].apply(function_clean)


In [41]:
df[['tweet_id', 'text', 'text_cleaned']]

,tweet_id,text,text_cleaned
0,119237,@AppleSupport causing the reply to be disregar...,applesupport caus repli disregard tap notif ke...
1,119238,@105835 Your business means a lot to us. Pleas...,105835 busi mean lot u plea dm name zip code a...
2,119239,@76328 I really hope you all change but I'm su...,76328 realli hope chang im sure wont dont
3,119240,@105836 LiveChat is online at the moment - htt...,105836 livechat onlin moment contact 03331 031...
4,119241,@VirginTrains see attached error message. I've...,virgintrain see attach error messag ive tri le...
...,...,...,...
88,119330,@105860 I wish Amazon had an option of where I...,105860 wish amazon option get ship up store av...
89,119331,They reschedule my shit for tomorrow https://t...,reschedul shit tomorrow
90,119332,"@105861 Hey Sara, sorry to hear of the issues ...",105861 hey sara sorri hear issu ask lay speed ...
91,119333,@Tesco bit of both - finding the layout cumber...,tesco bit find layout cumbersom remov item fav...


In [42]:
df.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id,text_lower,text_cleaned
0,119237,105834,True,Wed Oct 11 06:55:44 +0000 2017,@AppleSupport causing the reply to be disregar...,119236,NaN,@applesupport causing the reply to be disregar...,applesupport caus repli disregard tap notif ke...
1,119238,ChaseSupport,False,Wed Oct 11 13:25:49 +0000 2017,@105835 Your business means a lot to us. Pleas...,NaN,119239.0,@105835 your business means a lot to us. pleas...,105835 busi mean lot u plea dm name zip code a...
2,119239,105835,True,Wed Oct 11 13:00:09 +0000 2017,@76328 I really hope you all change but I'm su...,119238,NaN,@76328 i really hope you all change but i'm su...,76328 realli hope chang im sure wont dont
3,119240,VirginTrains,False,Tue Oct 10 15:16:08 +0000 2017,@105836 LiveChat is online at the moment - htt...,119241,119242.0,@105836 livechat is online at the moment - htt...,105836 livechat onlin moment contact 03331 031...
4,119241,105836,True,Tue Oct 10 15:17:21 +0000 2017,@VirginTrains see attached error message. I've...,119243,119240.0,@virgintrains see attached error message. i've...,virgintrain see attach error messag ive tri le...


In [43]:
df["text_cleaned"]

0     applesupport caus repli disregard tap notif ke...
1     105835 busi mean lot u plea dm name zip code a...
2             76328 realli hope chang im sure wont dont
3     105836 livechat onlin moment contact 03331 031...
4     virgintrain see attach error messag ive tri le...
                            ...                        
88    105860 wish amazon option get ship up store av...
89                              reschedul shit tomorrow
90    105861 hey sara sorri hear issu ask lay speed ...
91    tesco bit find layout cumbersom remov item fav...
92    105861 doesnt help plea dm full name address e...
Name: text_cleaned, Length: 93, dtype: object